![G4__4TRKCN_5BDKN_6@6TC3.png](https://i.loli.net/2021/10/09/QLamk34cBFdMNgr.png)

这里的feature map 有9行 表示有9个词，有6行 表示每个词用6维来进行编码
这里每个词向量是都是用word_embedding来表示

在这里面每个卷积核也不一定是相等的，例如第一个红色的卷积核，他的长度是6，宽是2，这样的好处就是可以同时考虑多个词，类似于bigram。卷积核设置为3或4都可以。

这里面的结构都很简单，难得是数据处理。


In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import torch.nn.functional as F

dtype = torch.FloatTensor
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
#定义一些常规的数据
#自己设置的数据集
#这里的实现没有用到ngram
sentences = ["i love you", "he loves me", "she likes baseball", "i hate you", "sorry for that", "this is awful"]
labels = [1, 1, 1, 0, 0, 0]  # 1 is good, 0 is not good. 二分类问题

# TextCNN Parameter
embedding_size = 2 #每个单词要用几维的向量来表示
sequence_length = len(sentences[0]) # 规定了每个句子的长度
num_classes = len(set(labels))  # 去重之后的长度
batch_size = 3

word_list = " ".join(sentences).split()
vocab = list(set(word_list))
word2idx = {w: i for i, w in enumerate(vocab)}
vocab_size = len(vocab)

In [4]:
#dataloader 调用数据的方式
def make_data(sentences, labels):
    inputs = []
    #下面把输入处理成6*3的矩阵 6句话 每句话3个词
    for sen in sentences:
        inputs.append([word2idx[n] for n in sen.split()]) 
    #print(inputs)
    targets = []
    for out in labels:
        targets.append(out)
    return inputs,targets
    
input_batch, target_batch = make_data(sentences,labels)
input_batch, target_batch = torch.LongTensor(input_batch),torch.LongTensor(target_batch)

dataset = Data.TensorDataset(input_batch, target_batch)
loader = Data.DataLoader(dataset, batch_size, True)

![PVCB_`SA00HOZ6I_1YVG_UI.png](https://i.loli.net/2021/10/10/34gdxsGkif9QJ2o.png)

输入数据是个矩阵，矩阵维度为 [batch_size, seqence_length]，矩阵的每一行是一个句子，每个数字代表一个词，然后我们把他映射到embedding层上，每个词用一个向量表示，比方说 12 可能会变成 [0.3,0.6,0.12,...]，因此整个数据无形中就增加了一个维度，变成了 [batch_size, sequence_length, embedding_size]

之后使用 unsqueeze(1) 函数使数据增加一个维度，变成 [batch_size, 1, sequence_length, embedding_size]。现在的数据才能做卷积，因为在传统 CNN 中，输入数据就应该是 [batch_size, in_channel, height, width] 这种维度

In [10]:
class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN,self).__init__()
        self.W =nn.Embedding(vocab_size,embedding_size)
        output_channel = 3
        self.conv = nn.Sequential(
            #conv [input_channel(=1), outputr_channel, (filter_height,filter_width),stride=1]
            #这里的filter height可以自己定义
            nn.Conv2d(1,output_channel,(2,embedding_size)),
            nn.ReLU(),
            # [pool]((filter_height,filter_width))
            nn.MaxPool2d((2,1))
        )
        #fc 
        self.fc = nn.Linear(output_channel, num_classes)
        
    def forward(self,X):
        '''
        X [batch_size, seq_len]
        '''
        batch_size = X.shape[0]
        emb_X = self.W(X) # [batch_size, sequence_length, embedding_size]
        #下一步在CV里用的比较多
        emb_X = emb_X.unsqueeze(1) # add channel(=1) [batch, channel(=1), sequence_length, embedding_size]
        conved = self .conv(emb_X)
        flatten = conved.view(batch_size, -1)
        output = self.fc(flatten)
        return output

In [11]:
model = TextCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 1e-3)

In [12]:
#Training 
for epoch in range(50000):
    for batch_x,batch_y in loader:
        pred = model(batch_x)
        loss = criterion(pred,batch_y)
        if (epoch + 1) % 1000 == 0:
            print('Epoch:', '%04d' % (epoch + 1), 'loss =', '{:.6f}'.format(loss))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

E:\anaconda\envs\pytorch\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 1000 loss = 0.004269
Epoch: 1000 loss = 0.001656
Epoch: 2000 loss = 0.000403
Epoch: 2000 loss = 0.000367
Epoch: 3000 loss = 0.000088
Epoch: 3000 loss = 0.000105
Epoch: 4000 loss = 0.000025
Epoch: 4000 loss = 0.000031
Epoch: 5000 loss = 0.000010
Epoch: 5000 loss = 0.000007
Epoch: 6000 loss = 0.000003
Epoch: 6000 loss = 0.000003
Epoch: 7000 loss = 0.000001
Epoch: 7000 loss = 0.000001
Epoch: 8000 loss = 0.000000
Epoch: 8000 loss = 0.000000
Epoch: 9000 loss = 0.000000
Epoch: 9000 loss = 0.000000
Epoch: 10000 loss = 0.000000
Epoch: 10000 loss = 0.000000
Epoch: 11000 loss = 0.000000
Epoch: 11000 loss = 0.000000
Epoch: 12000 loss = 0.000000
Epoch: 12000 loss = 0.000000
Epoch: 13000 loss = 0.000000
Epoch: 13000 loss = 0.000000
Epoch: 14000 loss = 0.000000
Epoch: 14000 loss = 0.000000
Epoch: 15000 loss = 0.000000
Epoch: 15000 loss = 0.000000
Epoch: 16000 loss = 0.000000
Epoch: 16000 loss = 0.000000
Epoch: 17000 loss = 0.000000
Epoch: 17000 loss = 0.000000
Epoch: 18000 loss = 0.000000
Epo

In [13]:
# Test
test_text = 'i hate me'
tests = [[word2idx[n] for n in test_text.split()]]
test_batch = torch.LongTensor(tests).to(device)
# Predict
model = model.eval()
predict = model(test_batch).data.max(1, keepdim=True)[1]
if predict[0][0] == 0:
    print(test_text,"is Bad Mean...")
else:
    print(test_text,"is Good Mean!!")

i hate me is Bad Mean...
